<a href="https://colab.research.google.com/github/hackcheek/rented-bike-prediction/blob/main/datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

import torch.nn as nn
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

**Extract data**

In [3]:
!git clone https://@github.com/hackcheek/rented-bike-prediction.git

Cloning into 'rented-bike-prediction'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 127 (delta 7), reused 30 (delta 4), pack-reused 89
Receiving objects: 100% (127/127), 32.70 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (36/36), done.


# Preprocessing

In [31]:
E_data_train = pd.read_excel('rented-bike-prediction/bike_train.xlsx')
E_data_test = pd.read_excel('rented-bike-prediction/bike_test.xlsx')

In [21]:
E_data_train.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt', 'dy'],
      dtype='object')

In [42]:
def preprocessing(data, n_components=None, no_consider=list()) -> pd.DataFrame:

    data['dy'] = data.dteday.dt.day

    features = {
       'season', 
       'yr', 
       'mnth', 
       'dy',
       'hr', 
       'holiday', 
       'weekday',
       'workingday', 
       'weathersit', 
       'temp', 
       'atemp', 
       'hum', 
       'windspeed',
    }
    features = list(features.difference(no_consider))
    print(features)

    # Get standarscaler 
    X = StandardScaler().fit_transform(data[features])

    # Data reductor with PCA
    components = n_components if n_components else int(len(features) / 2)
    pca = PCA(n_components=components)
    principal_components = pca.fit_transform(X)

    df = pd.DataFrame(
        principal_components,
        columns=[f'component {i}' for i in range(1, components+1)]
    )
    for i in no_consider

    # Add cnt for training data
    if 'cnt' in data.columns:
        df['cnt'] = data['cnt']

    
    return df


data_train = preprocessing(E_data_train, no_consider=['hr'])
data_test = preprocessing(E_data_test)

['mnth', 'weathersit', 'workingday', 'season', 'windspeed', 'weekday', 'atemp', 'hum', 'temp', 'dy', 'yr', 'holiday']


UFuncTypeError: ignored

In [40]:
data_train

,component 1,component 2,component 3,component 4,component 5,component 6,cnt
0,1.730224,-1.208046,0.538405,-0.869643,-1.537688,0.555955,16
1,1.823754,-1.248226,0.578924,-0.950797,-1.526791,0.539339,40
2,1.823754,-1.248226,0.578924,-0.950797,-1.526791,0.539339,32
3,1.769693,-1.030715,0.580155,-0.956218,-1.524848,0.550838,13
4,1.769693,-1.030715,0.580155,-0.956218,-1.524848,0.550838,1
...,...,...,...,...,...,...,...
11994,-0.032717,1.355336,-0.023069,0.598156,0.198304,0.102791,239
11995,0.166203,1.105275,0.788487,1.498271,-0.888533,-1.230675,170
11996,0.162046,0.807350,0.809441,1.466855,-1.336960,-1.636621,130
11997,0.162046,0.807350,0.809441,1.466855,-1.336960,-1.636621,98


# Deep learning

## Tran test split and get features and target on torch

In [1]:
X = data_train.drop('cnt', axis=1).to_numpy()
y = data_train['cnt'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

features = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)

NameError: ignored

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(13, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

# Random Forest

In [59]:
X = data_train.drop('cnt', axis=1)
y = data_train['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

features = X_train
target = y_train

In [64]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor()

param_grid = dict(
    max_depth = [10, 20, 25],
    max_features = ['auto'],
    n_estimators = [1000, 2000, 3000]
    # 'min_samples_leaf': [3, 4, 5],
    # 'min_samples_split': [8, 10, 12],
)
grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=4,
    n_jobs=-1, 
    verbose=2,
)


In [65]:
grid_search.fit(features, target)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

In [20]:
from sklearn.metrics import mean_squared_error

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    RMSE = np.sqrt(mean_squared_error(y_test, predictions, squared=False))
    print('Model Performance')
    print(f'Average Error: {np.mean(errors):0.4f} degrees.')
    print(f'Accuracy = {accuracy:0.2f}%.')
    print(f'RMSE: {RMSE}')
    return accuracy

In [56]:
best_rf = grid_search.best_estimator_
evaluate(best_rf, X_test, y_test)
best_rf

Model Performance
Average Error: 22.5680 degrees.
Accuracy = 64.52%.
RMSE: 6.236241765184201


RandomForestRegressor(max_depth=20, n_estimators=1000)

In [57]:
grid_search.best_estimator_.feature_importances_

array([0.01781223, 0.05791591, 0.01734068, 0.60151897, 0.00329621,
       0.01503694, 0.06030447, 0.02175945, 0.07748627, 0.0822528 ,
       0.0315493 , 0.01372677])

In [60]:
features

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
10213,1,1,3,15,0,2,1,1,0.34,0.3030,0.36,0.2985
8909,1,1,1,2,0,4,1,3,0.36,0.3485,0.93,0.2537
5767,3,0,9,18,0,5,1,2,0.68,0.6364,0.65,0.1940
8443,1,0,12,11,0,5,1,1,0.40,0.4091,0.58,0.4478
10922,2,1,4,6,0,4,1,1,0.36,0.3333,0.46,0.3284
...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1,0,3,12,0,0,0,1,0.36,0.3636,0.40,0.1045
2846,2,0,5,12,0,2,1,2,0.68,0.6364,0.61,0.4925
7513,4,0,11,16,0,1,1,1,0.62,0.6212,0.46,0.4179
4638,3,0,7,4,0,0,0,1,0.62,0.5909,0.73,0.1642


In [61]:
def deploy(model):
    pred = model.predict(data_test)
    pd.DataFrame({'pred': pred}).to_csv('hackcheek.csv')

In [62]:
deploy(best_rf)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2,1,5,4,0,0,0,1,0.52,0.5000,0.68,0.0896
1,2,1,5,5,0,0,0,1,0.50,0.4848,0.72,0.1045
2,2,1,5,6,0,0,0,1,0.50,0.4848,0.63,0.1343
3,2,1,5,7,0,0,0,1,0.52,0.5000,0.68,0.1940
4,2,1,5,8,0,0,0,1,0.56,0.5303,0.56,0.1642
...,...,...,...,...,...,...,...,...,...,...,...,...
5375,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642
5376,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642
5377,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642
5378,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343


# Naive bayes